<h1><center>Web Scrapping using Selenium</center></h1> 


 ### Importing all the required libraries

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

### Set the path of your chrome driver

In [20]:
chrome_path = r"C:\Users\KAK12\AppData\Local\Continuum\Anaconda3\Scripts\chromedriver.exe"

driver = webdriver.Chrome(chrome_path)

### The link you want to scrape(I am scrapping the LinkedIn website)

In [21]:
driver.get("https://www.linkedin.com/jobs/")

### Extracting the Domains in the job listing page

In [7]:
Domain=[]
for i in range(1,7):
	result = driver.find_element_by_xpath('//*[@id="content-outlet"]/section[2]/div[%s]/a/section/h4'%(i)).text
	Domain.append(result)

In [8]:
Domain

['Management Jobs',
 'Sales Jobs',
 'Engineering Jobs',
 'Information Technology Jobs',
 'Healthcare Jobs',
 'Finance Jobs']

### Extracting the link of each of the Domains page

In [9]:
link = []
for i in range(1,7):
	result1 = driver.find_element_by_xpath('//*[@id="content-outlet"]/section[2]/div[%s]/a'%(i)).get_attribute("href")
	link.append(result1)

In [10]:
link

['https://www.linkedin.com/jobs/management-jobs?trk=guest_jobs_homepage',
 'https://www.linkedin.com/jobs/sales-jobs?trk=guest_jobs_homepage',
 'https://www.linkedin.com/jobs/engineering-jobs?trk=guest_jobs_homepage',
 'https://www.linkedin.com/jobs/information-technology-jobs?trk=guest_jobs_homepage',
 'https://www.linkedin.com/jobs/healthcare-jobs?trk=guest_jobs_homepage',
 'https://www.linkedin.com/jobs/finance-jobs?trk=guest_jobs_homepage']

### Iterating through the link and grabbing the number of job postings specific to each domain

In [13]:
#Number of job postings
a=[]
for i in range(1,7): 
	driver.find_element_by_xpath('//*[@id="content-outlet"]/section[2]/div[%s]/div/a'%(i)).click()
	a.append(driver.find_element_by_xpath('//*[@id="content-outlet"]/div/section[3]/div[2]/div/section[1]/div/div[1]/div/div/strong').text)
	#To go back to the previous page
	driver.execute_script("window.history.go(-1)")


### Converting it to a dataframe 

In [14]:
job_listing_df = pd.DataFrame({'Domain' : Domain , 'Link' : link , 'No_of_Job_openings' : a})

In [15]:
job_listing_df

,Domain,Link,No_of_Job_openings
0,Management Jobs,https://www.linkedin.com/jobs/management-jobs?...,"2,029,520"
1,Sales Jobs,https://www.linkedin.com/jobs/sales-jobs?trk=g...,"1,260,888"
2,Engineering Jobs,https://www.linkedin.com/jobs/engineering-jobs...,"383,760"
3,Information Technology Jobs,https://www.linkedin.com/jobs/information-tech...,"73,357"
4,Healthcare Jobs,https://www.linkedin.com/jobs/healthcare-jobs?...,"584,168"
5,Finance Jobs,https://www.linkedin.com/jobs/finance-jobs?trk...,"188,505"


### Getting the input from the user about the job title and location and passing it on to the browser 

In [29]:
#Enter the Job_position
job_title = input("Enter a Job_title: ")
job_seach=driver.find_element_by_xpath('//*[@id="keyword-search-box"]').clear()
job_search=driver.find_element_by_xpath('//*[@id="keyword-search-box"]')
job_search.send_keys(job_title)

#Enter the location
job_locate = input("Enter a Job_location: ")
job_location=driver.find_element_by_xpath('//*[@id="location-search-box"]').clear()
job_location=driver.find_element_by_xpath('//*[@id="location-search-box"]')
job_location.send_keys(job_locate)

#Search
driver.find_element_by_xpath('//*[@id="content-outlet"]/section[1]/div/div/section/form/div/button').click()

Enter a Job_title: Data Analytics
Enter a Job_location: Dallas, Texas


### Getting the number of job openings for the specific job title and location

In [ ]:
driver.find_element_by_xpath('//*[@id="content-outlet"]/div/section[3]/div[2]/div/section[1]/div/div[1]/div/div/strong').text

### Feeding in the US_Cities and States Dataset

In [4]:
city_state = pd.read_csv("city_State_database.csv")

In [56]:
city_state.head()


,primary_city,state
0,Holtsville,NY
1,Adjuntas,PR
2,Aguada,PR
3,Aguadilla,PR
4,Maricao,PR


In [14]:
city_state['Location'] = city_state['primary_city'] + ", " +city_state['state']

### Passing in the Location value to grab the count of job openings but US cities and states for Data Analytics position

In [ ]:
job_posts = []
for i in city_state['Location'].iloc[0:29785]:
    
    time.sleep(30)
    
    job_seach=driver.find_element_by_xpath('//*[@id="keyword-search-box"]').clear()
    job_search=driver.find_element_by_xpath('//*[@id="keyword-search-box"]')
    job_search.send_keys('Data Analytics')
    
    job_location=driver.find_element_by_xpath('//*[@id="location-search-box"]').clear()
    job_location=driver.find_element_by_xpath('//*[@id="location-search-box"]')
    job_location.send_keys(i)
    
    driver.find_element_by_xpath('//*[@id="content-outlet"]/section[1]/div/div/section/form/div/button').click()
    job_posts.append(driver.find_element_by_xpath('//*[@id="content-outlet"]/div/section[3]/div[2]/div/section[1]/div/div[1]/div/div/strong').text)
    driver.execute_script("window.history.go(-1)")

In [ ]:
city_state_sample = city_state[0:29786]
city_state_sample['Job_postings'] = job_posts

## A Dataframe which shows the count of Data Analytics job postings by location

In [76]:
city_state_sample

,primary_city,state,Location,Job_postings
0,Holtsville,NY,"Holtsville, NY",54
1,Adjuntas,PR,"Adjuntas, PR","2,370"
2,Aguada,PR,"Aguada, PR",717
3,Aguadilla,PR,"Aguadilla, PR",756
4,Maricao,PR,"Maricao, PR","1,354"
